In [2]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Conv2D, MaxPooling2D, Dropout
from tensorflow.keras.datasets import mnist

In [3]:
# The dataset is already pre-split into training and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

In [4]:
# Normalize pixel values to range between 0 and 1 for better model performance
x_train, x_test = x_train / 255.0, x_test / 255.0

In [5]:
# Reshape data to add an additional dimension (28x28x1) for CNN input
# The '1' here represents the single grayscale color channel
x_train = x_train.reshape(-1, 28, 28, 1)
x_test = x_test.reshape(-1, 28, 28, 1)

In [6]:
# Define the CNN model
def create_cnn():
    model = Sequential([
        Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape=(28, 28, 1)),
        MaxPooling2D(pool_size=(2, 2)),
        Conv2D(64, kernel_size=(3, 3), activation='relu'),
        MaxPooling2D(pool_size=(2, 2)),
        Flatten(),
        Dense(128, activation='relu'),
        Dropout(0.5),
        Dense(10, activation='softmax')
    ])
    return model


In [7]:
#Create and compile the model
model = create_cnn()

C:\Users\naman\AppData\Roaming\Python\Python312\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


In [8]:
# Compile the model with Adam optimizer, sparse categorical crossentropy loss, and accuracy metric
# Adam optimizer adapts learning rate dynamically, and sparse_categorical_crossentropy is ideal for integer labels
model.compile(optimizer='adam', loss='sparse_categorical_crossentropy', metrics=['accuracy'])

In [9]:
# Train the model on the training data with validation split of 20%
# Using 10 epochs and a batch size of 32
history = model.fit(x_train, y_train, epochs=10, batch_size=32, validation_split=0.2)

Epoch 1/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.8503 - loss: 0.4691 - val_accuracy: 0.9820 - val_loss: 0.0589
Epoch 2/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9721 - loss: 0.0919 - val_accuracy: 0.9868 - val_loss: 0.0453
Epoch 3/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9799 - loss: 0.0661 - val_accuracy: 0.9892 - val_loss: 0.0377
Epoch 4/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9839 - loss: 0.0541 - val_accuracy: 0.9883 - val_loss: 0.0400
Epoch 5/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9865 - loss: 0.0403 - val_accuracy: 0.9912 - val_loss: 0.0354
Epoch 6/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9888 - loss: 0.0358 - val_accuracy: 0.9904 - val_loss: 0.0364
Epoch 7/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9905 - loss: 0.0297 - val_accuracy: 0.9909 - val_loss: 0.0354
Epoch 8/10
1500/1500 ━━━━━━━━━━━━━━━━━━━━ 6s 4ms/step - accuracy: 0.9921 - loss: 0.0242 - 

In [10]:
# Test the model on the test data to get accuracy and loss values
test_loss, test_accuracy = model.evaluate(x_test, y_test)
print(f"Test accuracy: {test_accuracy}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - accuracy: 0.9900 - loss: 0.0328
Test accuracy: 0.9922999739646912


In [ ]:
# Save the model as an H5 file
model.save('my.h5')


In [ ]:
import cv2
import numpy as np
from tensorflow.keras.models import load_model

# Load your trained model
model = load_model('my.h5')

def preprocess_frame(frame):
    """Preprocess the frame for the model: grayscale, resize, normalize."""
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    resized = cv2.resize(gray, (28, 28))
    resized = cv2.bitwise_not(resized)  # Invert for MNIST-like data
    normalized = resized / 255.0
    input_data = normalized.reshape(1, 28, 28, 1)
    return input_data

def predict_digit(frame):
    """Use the model to predict the digit from a webcam frame."""
    input_data = preprocess_frame(frame)
    prediction = model.predict(input_data)
    digit = np.argmax(prediction)  # Get the highest probability digit
    return digit

def display_message(message, frame, pause_time=2000):
    """Display the message on the webcam frame for a short period."""
    cv2.putText(frame, message, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 2, (0, 255, 0), 4)
    cv2.imshow('Digit Recognition', frame)
    cv2.waitKey(pause_time)

def run_digit_recognition():
    """Run the webcam loop to predict digits and stop when the correct digit is predicted."""
    cap = cv2.VideoCapture(0)

    print("Starting digit recognition... Press 'q' to quit at any time.")

    while True:
        ret, frame = cap.read()
        if not ret:
            print("Failed to capture the frame. Exiting...")
            break

        # Predict the digit
        digit = predict_digit(frame)

        # Display the prediction on the video feed
        cv2.putText(frame, f"Predicted Digit: {digit}", (10, 30), 
                    cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
        cv2.imshow('Digit Recognition', frame)

        # Check if 'q' is pressed to quit
        if cv2.waitKey(1) & 0xFF == ord('q'):
            print("Exiting digit recognition.")
            break

    cap.release()
    cv2.destroyAllWindows()

def main():
    """Main function that controls the digit recognition process."""
    while True:
        # Start the digit recognition task
        run_digit_recognition()

        # After task completion, ask if the user wants to restart
        print("\nPrediction completed. Press 'r' to restart or 'q' to quit.")
        key = cv2.waitKey(0) & 0xFF
        if key == ord('r'):
            print("Restarting the digit recognition task...\n")
        elif key == ord('q'):
            print("Exiting program. Goodbye!")
            break

if __name__ == "__main__":
    main()


Starting digit recognition... Press 'q' to quit at any time.
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 44ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 15ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 14ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 13ms/step
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s

KeyboardInterrupt: 